In [1]:
import pandas as pd
import numpy as np

import cenpy as cen
from collections import OrderedDict

import time # to make program sleep temporarily 

import warnings
warnings.simplefilter('ignore')

/Users/lmhartmann/miniconda3/lib/python3.9/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
ny_zip_ref = pd.read_csv('New_York_State_Zip_Codes-County_FIPS_Cross-Reference.csv')

# only need to keep 'County Name' and 'ZIP Code'

ny_zip_ref = ny_zip_ref[['County Name', 'ZIP Code']]
ny_zip_ref

,County Name,ZIP Code
0,Albany,12007
1,Albany,12009
2,Albany,12023
3,Albany,12041
4,Albany,12045
...,...,...
2538,Yates,14561
2539,Yates,14837
2540,Yates,14842
2541,Yates,14857


In [3]:
# might not need this

sites = pd.read_csv('Voting_Poll_Sites.csv')
sites

,BOROUGH,SITE_STATUS,SITE_NAME,SITE_NUMBER,STREET_NUMBER,STREET_SUFFIX,STREET_NAME,POSTCODE,CITY,VOTER_ENTRANCE,HANDICAP_ENTRANCE,Latitude,Longitude,Community Board,Council District,Census Tract,BIN,BBL,NTA,Location
0,BROOKLYN,A,PS 12,11537,430,NaN,Howard Avenue,11233,Brooklyn,430 Howard Avenue (not used for voters),Enter on Prospect Place through school yard,NaN,NaN,16.0,41.0,363.0,3039174.0,3.014570e+09,Ocean Hill,NaN
1,NaN,A,Wyatt T. Walker Senior Housing,11517,2177,NaN,Frederick Douglass Boulevard,10026,New York,2177 Frederick Douglas Blvd,2177 Frederick Douglas Blvd,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,QUEENS,A,Allen AME Senior Center,10590,112-04,NaN,167 Street,11433,Jamaica,112-04 167th Street,112-04 167th Street,40.691478,-73.783335,12.0,27.0,266.0,4435291.0,4.123220e+09,South Jamaica,"(40.691478, -73.783335)"
3,BROOKLYN,A,Marlboro Memorial Post,11493,300,NaN,Avenue X,11223,Brooklyn,300 Avenue X,300 Avenue X,NaN,NaN,15.0,47.0,37402.0,3195077.0,3.071940e+09,Gravesend,NaN
4,BROOKLYN,A,PAL-Wynn Center,11629,495,NaN,Gates Avenue,11216,Brooklyn,495 Gates Avenue,495 Gates Avenue,NaN,NaN,3.0,36.0,265.0,3050974.0,3.018090e+09,Bedford,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1226,QUEENS,A,PS 107-Thomas A Dooley,Q0082,167-02,NaN,45 Avenue,11358,Flushing,167-02 45 Avenue,Side Entrance facing 168th Street near 45th Av...,40.755951,-73.799746,7.0,19.0,1195.0,4123454.0,4.054460e+09,East Flushing,"(40.755951, -73.799746)"
1227,MANHATTAN,A,Triangle House,M0015,112,NaN,East 128 Street,10035,New York,112 East 128 Street,112 East 128 Street,40.806665,-73.937085,11.0,9.0,242.0,1054499.0,1.017750e+09,East Harlem North,"(40.806665, -73.937085)"
1228,BROOKLYN,A,Warbasse Houses Bldg 5C,B0274,545,NaN,Neptune Avenue,11224,Brooklyn,545 Neptune Avenue,West Avenue,40.579987,-73.973001,13.0,47.0,354.0,3257102.0,3.072500e+09,West Brighton,"(40.579987, -73.973001)"
1229,BRONX,A,Pelham Pkwy Community Center,00036,785,NaN,Pelham Parkway North,10467,Bronx,785 Pelham Parkway North - around playgound,785 Pelham Parkway North - around playgound,40.857972,-73.864564,11.0,13.0,328.0,2093364.0,2.043490e+09,Bronxdale,"(40.857972, -73.864564)"


In [4]:
pres = pd.read_csv('2016President.csv')

# keep select columns
pres_cols = ["county", "office", "party", "candidate", "votes"]
pres = pres[pres_cols]

# keep only where office == President
pres = pres.loc[pres['office'] == "President"]
pres = pres.drop(['office'], axis = 1)

# drop NAN values
pres = pres.dropna()

# get voting sums
agg_functions = {'county': 'first', 'candidate': 'first', 'party': 'first', 'votes': 'sum'}
pres_agg = pres.groupby(['county', 'candidate']).aggregate(agg_functions)

# drop index
pres_agg = pres_agg.reset_index(drop = True)
pres_agg

pres_agg

,county,candidate,party,votes
0,Albany,Donald J. Trump,REP,47808.0
1,Albany,Gary Johnson,IND,4792.0
2,Albany,Hillary Clinton,DEM,83071.0
3,Albany,Jill Stein,GRN,2475.0
4,Allegany,Donald J. Trump,REP,12525.0
...,...,...,...,...
247,Wyoming,Jill Stein,GRN,180.0
248,Yates,Donald J. Trump,REP,5660.0
249,Yates,Gary Johnson,IND,494.0
250,Yates,Hillary Clinton,DEM,3659.0


In [26]:
counties = pres_agg.county.unique().tolist()
counties
# print(len(counties)) # 63

['Albany',
 'Allegany',
 'Bronx',
 'Broome',
 'Cattaraugus',
 'Cayuga',
 'Chautauqua',
 'Chemung',
 'Chenango',
 'Clinton',
 'Columbia',
 'Cortland',
 'Delaware',
 'Dutchess',
 'Erie',
 'Essex',
 'Franklin',
 'Fulton',
 'Genesee',
 'Greene',
 'Hamilton',
 'Herkimer',
 'Jefferson',
 'Kings',
 'Lewis',
 'Livingston',
 'Madison',
 'Monroe',
 'Montgomery',
 'Nassau',
 'New York',
 'Niagara',
 'Oneida',
 'Onondaga',
 'Ontario',
 'Orange',
 'Orleans',
 'Oswego',
 'Otsego',
 'Putnam',
 'Queens',
 'Rensselaer',
 'Richmond',
 'Rockland',
 'Saratoga',
 'Schenectady',
 'Schoharie',
 'Schuyler',
 'Seneca',
 'St. Lawrence',
 'Steuben',
 'Suffolk',
 'Sullivan',
 'Tioga',
 'Tompkins',
 'Total',
 'Ulster',
 'Warren',
 'Washington',
 'Wayne',
 'Westchester',
 'Wyoming',
 'Yates']

In [6]:
# using cenpy get NY data
acs = cen.products.ACS()
acs.from_place('NY')

Matched: NY to Onyx CDP within layer Census Designated Places


,geometry,state,county,tract,GEOID


In [7]:
# to see what categories we can look through
acs.tables

,description,columns
table_name,,
B01001,SEX BY AGE,"[B01001_001E, B01001_002E, B01001_003E, B01001..."
B01002,MEDIAN AGE BY SEX,"[B01002_001E, B01002_002E, B01002_003E]"
B01003,TOTAL POPULATION,[B01003_001E]
B02001,RACE,"[B02001_001E, B02001_002E, B02001_003E, B02001..."
B02008,WHITE ALONE OR IN COMBINATION WITH ONE OR MORE...,[B02008_001E]
...,...,...
C27014,PUBLIC HEALTH INSURANCE BY WORK EXPERIENCE,"[C27014_001E, C27014_002E, C27014_003E, C27014..."
C27016,HEALTH INSURANCE COVERAGE STATUS BY RATIO OF I...,"[C27016_001E, C27016_002E, C27016_003E, C27016..."
C27017,PRIVATE HEALTH INSURANCE BY RATIO OF INCOME TO...,"[C27017_001E, C27017_002E, C27017_003E, C27017..."


In [ ]:
# all_options = acs.tables.description.unique().tolist()
# all_options 

# # ones to look at 
# # sex
# # race
# # education
# # health insurance
# # family income

## Create Dataframe on Sex Data

In [8]:
acs.filter_tables('SEX', by = 'description')

,description,columns
table_name,,
B01001,SEX BY AGE,"[B01001_001E, B01001_002E, B01001_003E, B01001..."
B01002,MEDIAN AGE BY SEX,"[B01002_001E, B01002_002E, B01002_003E]"
B05003,SEX BY AGE BY NATIVITY AND CITIZENSHIP STATUS,"[B05003_001E, B05003_002E, B05003_003E, B05003..."
B05004,MEDIAN AGE BY NATIVITY AND CITIZENSHIP STATUS ...,"[B05004_001E, B05004_002E, B05004_003E, B05004..."
B05008,SEX BY PLACE OF BIRTH BY YEAR OF ENTRY FOR THE...,"[B05008_001E, B05008_002E, B05008_003E, B05008..."
...,...,...
C27005,DIRECT-PURCHASE HEALTH INSURANCE BY SEX BY AGE,"[C27005_001E, C27005_002E, C27005_003E, C27005..."
C27006,MEDICARE COVERAGE BY SEX BY AGE,"[C27006_001E, C27006_002E, C27006_003E, C27006..."
C27007,MEDICAID/MEANS-TESTED PUBLIC COVERAGE BY SEX B...,"[C27007_001E, C27007_002E, C27007_003E, C27007..."


In [9]:
totalSexData = acs.filter_variables('B01001')

In [27]:
# sexData = []

# the columns we are looking for
to_keep = ["Estimate!!Total:", "Estimate!!Total:!!Female:",
          "Estimate!!Total:!!Female:!!18 and 19 years", "Estimate!!Total:!!Female:!!20 to 24 years",
          "Estimate!!Total:!!Female:!!25 to 29 years", "Estimate!!Total:!!Female:!!30 to 34 years",
          "Estimate!!Total:!!Female:!!35 to 44 years", "Estimate!!Total:!!Female:!!45 to 54 years",
          "Estimate!!Total:!!Female:!!55 to 64 years", "Estimate!!Total:!!Female:!!65 to 74 years",
          "Estimate!!Total:!!Female:!!75 to 84 years", "Estimate!!Total:!!Female:!!85 years and over",
          "Estimate!!Total:!!Male:",
          "Estimate!!Total:!!Male:!!18 and 19 years", "Estimate!!Total:!!Male:!!20 to 24 years",
          "Estimate!!Total:!!Male:!!25 to 29 years", "Estimate!!Total:!!Male:!!30 to 34 years",
          "Estimate!!Total:!!Male:!!35 to 44 years", "Estimate!!Total:!!Male:!!45 to 54 years",
          "Estimate!!Total:!!Male:!!55 to 64 years", "Estimate!!Total:!!Male:!!65 to 74 years",
          "Estimate!!Total:!!Male:!!75 to 84 years", "Estimate!!Total:!!Male:!!85 years and over"]

# renamed column names
rename = ["Total:", "Total Female:",
          "Total Female: 18 and 19 years", "Total Female: 20 to 24 years",
          "Total Female: 25 to 29 years", "Total Female: 30 to 34 years",
          "Total Female: 35 to 44 years", "Total Female: 45 to 54 years",
          "Total Female: 55 to 64 years", "Total Female: 65 to 74 years",
          "Total Female: 75 to 84 years", "Total Female: 85 years and over",
          "Total Male:",
          "Total Male: 18 and 19 years", "Total Male: 20 to 24 years",
          "Total Male: 25 to 29 years", "Total Male: 30 to 34 years",
          "Total Male: 35 to 44 years", "Total Male: 45 to 54 years",
          "Total Male: 55 to 64 years", "Total Male: 65 to 74 years",
          "Total Male: 75 to 84 years", "Total Male: 85 years and over"]

# do the first 20 counties
for i in range(50, 63, 1):

    name = counties[i] + ", NY"
    currentDF = acs.from_county(name, variables = 'B01001')
    current = counties[i]
    
    # if first round through
    # create labels to rename the dataframes
    if i == 0:
        indexVals = totalSexData.index
        labels = totalSexData['label'].to_list()
        labelsDict = OrderedDict({})
        for key in indexVals:
            for value in labels:
                labelsDict[key] = value
                labels.remove(value)
                break

        labelsDict.update({"geometry": "geometry"})
        labelsDict.move_to_end("geometry", last = False)

        labelsDict.update({"GEOID": "GEOID"})
        labelsDict.move_to_end("GEOID", last = False)

        labelsDict.update({"state": "state"})
        labelsDict.update({"county": "county"})
        labelsDict.update({"tract": "tract"})
        labelsDict.update({"NAME": "NAME"})
    
    # rename the dataframe
    currentDF.columns = currentDF.columns.to_series().map(labelsDict)
    
    # drop the duplciate columns in the county dataframe
    currentDF = currentDF.loc[:, ~currentDF.columns.duplicated()].copy()
    
    # only keep the previously selected columns
    currentDF = currentDF[to_keep]

    # rename the columns 
    currentDF = currentDF.set_axis(rename, axis = 1)
    
    
    # get the sums for each category and put into a dictonary
    current_sums = currentDF.sum().to_dict()
    current_sums = OrderedDict(current_sums)

    current_sums.update({"County": current})
    current_sums.move_to_end("County", last = False)

    current_sums = dict(current_sums)

    # if first run through
    # get the headers for the final dataframe
    # append to the final dataframe
    if i == 0:
        headers = list(current_sums.keys())
        sexData.append(headers)
        
    # create inner_row to append
    # and append to final dataframe
    inner_row = list(current_sums.values())
    sexData.append(inner_row)
    
    if (i % 2 == 0):
        print(inner_row)
    
    
    time.sleep(5)
    

['Steuben', 91286.0, 46006.0, 905.0, 2511.0, 2579.0, 2535.0, 5125.0, 6270.0, 6926.0, 5313.0, 3061.0, 1366.0, 45280.0, 993.0, 2591.0, 2638.0, 2598.0, 5153.0, 6233.0, 6968.0, 4900.0, 2534.0, 728.0]
['Sullivan', 59949.0, 29836.0, 466.0, 1548.0, 1651.0, 1744.0, 3155.0, 4164.0, 4822.0, 3805.0, 1980.0, 704.0, 30113.0, 630.0, 1623.0, 1685.0, 1685.0, 3142.0, 4225.0, 4913.0, 3777.0, 1939.0, 384.0]
['Tompkins', 82359.0, 42174.0, 3990.0, 5298.0, 2687.0, 2322.0, 4498.0, 4718.0, 5621.0, 4202.0, 1983.0, 1149.0, 40185.0, 3701.0, 5682.0, 2797.0, 2289.0, 4071.0, 4506.0, 4997.0, 3881.0, 1580.0, 575.0]
['Ulster', 151001.0, 77592.0, 2103.0, 4481.0, 4547.0, 4199.0, 8375.0, 11143.0, 12860.0, 9925.0, 5372.0, 2441.0, 73409.0, 1852.0, 4528.0, 4704.0, 4193.0, 8318.0, 10722.0, 12288.0, 8862.0, 3904.0, 1525.0]
['Washington', 57686.0, 28690.0, 487.0, 1435.0, 1643.0, 1620.0, 3198.0, 4292.0, 4569.0, 3356.0, 1838.0, 944.0, 28996.0, 589.0, 1653.0, 1739.0, 1710.0, 3429.0, 4292.0, 4538.0, 3281.0, 1582.0, 438.0]
['Westch

In [28]:
# make into dataframe
sexDF = pd.DataFrame(sexData)

# make first row the header
new_header = sexDF.iloc[0] 
sexDF = sexDF[1:] 
sexDF.columns = new_header 

# view
sexDF

,County,Total:,Total Female:,Total Female: 18 and 19 years,Total Female: 20 to 24 years,Total Female: 25 to 29 years,Total Female: 30 to 34 years,Total Female: 35 to 44 years,Total Female: 45 to 54 years,Total Female: 55 to 64 years,...,Total Male: 18 and 19 years,Total Male: 20 to 24 years,Total Male: 25 to 29 years,Total Male: 30 to 34 years,Total Male: 35 to 44 years,Total Male: 45 to 54 years,Total Male: 55 to 64 years,Total Male: 65 to 74 years,Total Male: 75 to 84 years,Total Male: 85 years and over
1,Albany,231246.0,118927.0,4517.0,11264.0,7148.0,6872.0,12631.0,15372.0,17789.0,...,4465.0,11548.0,7392.0,7327.0,12849.0,14959.0,16516.0,11693.0,5090.0,2276.0
2,Allegany,44565.0,22044.0,1124.0,1768.0,1065.0,1169.0,2258.0,2676.0,3212.0,...,1334.0,2020.0,1143.0,1124.0,2248.0,2629.0,3301.0,2479.0,1088.0,481.0
3,Bronx,315533.0,164059.0,4405.0,10370.0,10733.0,10563.0,17809.0,20292.0,20985.0,...,4395.0,10760.0,11539.0,10584.0,18655.0,18770.0,18439.0,14595.0,8660.0,3521.0
4,Broome,165108.0,84187.0,2909.0,6913.0,4423.0,4335.0,8417.0,10583.0,12909.0,...,3187.0,7421.0,4688.0,4502.0,8310.0,10243.0,12542.0,8471.0,4591.0,1747.0
5,Cattaraugus,70859.0,35715.0,947.0,2079.0,1891.0,1879.0,3841.0,4776.0,5603.0,...,982.0,2146.0,1944.0,1881.0,3821.0,4758.0,5546.0,3976.0,1898.0,567.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,Washington,57686.0,28690.0,487.0,1435.0,1643.0,1620.0,3198.0,4292.0,4569.0,...,589.0,1653.0,1739.0,1710.0,3429.0,4292.0,4538.0,3281.0,1582.0,438.0
60,Wayne,84123.0,42635.0,800.0,2311.0,2257.0,2355.0,4641.0,6345.0,6843.0,...,973.0,2313.0,2359.0,2316.0,4596.0,6165.0,6579.0,4612.0,2126.0,618.0
61,Westchester,622855.0,319393.0,7715.0,17738.0,14681.0,15676.0,34912.0,47238.0,47897.0,...,7672.0,18240.0,14559.0,15990.0,35153.0,45566.0,46094.0,30927.0,15935.0,7845.0
62,Wyoming,36882.0,17947.0,333.0,975.0,1067.0,1005.0,1967.0,2589.0,2788.0,...,386.0,1062.0,1250.0,1202.0,2244.0,2847.0,2976.0,2103.0,910.0,255.0


In [29]:
# write to sex data to csv
# as backup
sexDF.to_csv("NY_sexData.csv")

In [36]:
# check csv saved correctly
# read data in from csv
# this is so you don't have to run through the previous for loop (saves time)

sexDF = pd.read_csv('NY_sexData.csv')
sexDF = sexDF.drop(['Unnamed: 0'], axis = 1)
sexDF

,County,Total:,Total Female:,Total Female: 18 and 19 years,Total Female: 20 to 24 years,Total Female: 25 to 29 years,Total Female: 30 to 34 years,Total Female: 35 to 44 years,Total Female: 45 to 54 years,Total Female: 55 to 64 years,...,Total Male: 18 and 19 years,Total Male: 20 to 24 years,Total Male: 25 to 29 years,Total Male: 30 to 34 years,Total Male: 35 to 44 years,Total Male: 45 to 54 years,Total Male: 55 to 64 years,Total Male: 65 to 74 years,Total Male: 75 to 84 years,Total Male: 85 years and over
0,Albany,231246.0,118927.0,4517.0,11264.0,7148.0,6872.0,12631.0,15372.0,17789.0,...,4465.0,11548.0,7392.0,7327.0,12849.0,14959.0,16516.0,11693.0,5090.0,2276.0
1,Allegany,44565.0,22044.0,1124.0,1768.0,1065.0,1169.0,2258.0,2676.0,3212.0,...,1334.0,2020.0,1143.0,1124.0,2248.0,2629.0,3301.0,2479.0,1088.0,481.0
2,Bronx,315533.0,164059.0,4405.0,10370.0,10733.0,10563.0,17809.0,20292.0,20985.0,...,4395.0,10760.0,11539.0,10584.0,18655.0,18770.0,18439.0,14595.0,8660.0,3521.0
3,Broome,165108.0,84187.0,2909.0,6913.0,4423.0,4335.0,8417.0,10583.0,12909.0,...,3187.0,7421.0,4688.0,4502.0,8310.0,10243.0,12542.0,8471.0,4591.0,1747.0
4,Cattaraugus,70859.0,35715.0,947.0,2079.0,1891.0,1879.0,3841.0,4776.0,5603.0,...,982.0,2146.0,1944.0,1881.0,3821.0,4758.0,5546.0,3976.0,1898.0,567.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,Washington,57686.0,28690.0,487.0,1435.0,1643.0,1620.0,3198.0,4292.0,4569.0,...,589.0,1653.0,1739.0,1710.0,3429.0,4292.0,4538.0,3281.0,1582.0,438.0
59,Wayne,84123.0,42635.0,800.0,2311.0,2257.0,2355.0,4641.0,6345.0,6843.0,...,973.0,2313.0,2359.0,2316.0,4596.0,6165.0,6579.0,4612.0,2126.0,618.0
60,Westchester,622855.0,319393.0,7715.0,17738.0,14681.0,15676.0,34912.0,47238.0,47897.0,...,7672.0,18240.0,14559.0,15990.0,35153.0,45566.0,46094.0,30927.0,15935.0,7845.0
61,Wyoming,36882.0,17947.0,333.0,975.0,1067.0,1005.0,1967.0,2589.0,2788.0,...,386.0,1062.0,1250.0,1202.0,2244.0,2847.0,2976.0,2103.0,910.0,255.0


## Create Dataframe on Race Data

In [39]:
acs.filter_tables('RACE', by = 'description')

,description,columns
table_name,,
B02001,RACE,"[B02001_001E, B02001_002E, B02001_003E, B02001..."
B02008,WHITE ALONE OR IN COMBINATION WITH ONE OR MORE...,[B02008_001E]
B02009,BLACK OR AFRICAN AMERICAN ALONE OR IN COMBINAT...,[B02009_001E]
B02010,AMERICAN INDIAN AND ALASKA NATIVE ALONE OR IN ...,[B02010_001E]
B02011,ASIAN ALONE OR IN COMBINATION WITH ONE OR MORE...,[B02011_001E]
B02012,NATIVE HAWAIIAN AND OTHER PACIFIC ISLANDER ALO...,[B02012_001E]
B02013,SOME OTHER RACE ALONE OR IN COMBINATION WITH O...,[B02013_001E]
B03002,HISPANIC OR LATINO ORIGIN BY RACE,"[B03002_001E, B03002_002E, B03002_003E, B03002..."
B25006,RACE OF HOUSEHOLDER,"[B25006_001E, B25006_002E, B25006_003E, B25006..."


In [43]:
totalRaceData = acs.filter_variables('B02001')
totalRaceData["label"].unique().tolist()

['Estimate!!Total:!!Two or more races:!!Two races including Some other race',
 'Estimate!!Total:!!Some other race alone',
 'Estimate!!Total:!!Two or more races:',
 'Estimate!!Total:!!Two or more races:!!Two races excluding Some other race, and three or more races',
 'Estimate!!Total:!!White alone',
 'Estimate!!Total:',
 'Estimate!!Total:!!Native Hawaiian and Other Pacific Islander alone',
 'Estimate!!Total:!!Asian alone',
 'Estimate!!Total:!!American Indian and Alaska Native alone',
 'Estimate!!Total:!!Black or African American alone']

In [57]:
# raceData = []

to_keep = ['Estimate!!Total:!!Two or more races:!!Two races including Some other race', 
           "Estimate!!Total:!!Some other race alone", 
           "Estimate!!Total:!!Two or more races:", 
           'Estimate!!Total:!!Two or more races:!!Two races excluding Some other race, and three or more races',
           "Estimate!!Total:!!White alone", "Estimate!!Total:",
           "Estimate!!Total:!!Native Hawaiian and Other Pacific Islander alone",
           "Estimate!!Total:!!Asian alone", 
           "Estimate!!Total:!!American Indian and Alaska Native alone",
           "Estimate!!Total:!!Black or African American alone"]

# renamed column names
rename = ["Total: Two or more races: Two races including Some other race", "Total: Some other race alone", 
          "Total: Two or more races", "Total: Two or more races excluding Some other race, and three or more races",
          "Total: White alone", "Total:",
          "Total: Native Hawaiian and Other Pacific Islander alone",
          "Total: Asian alone", "Total: American Indian and Alaska Native alone",
          "Total: Black or African American alone"]


for i in range(30, 63, 1):

    name = counties[i] + ", NY"
    currentDF = acs.from_county(name, variables = 'B02001')
    current = counties[i]
    
    # if first round through
    # create labels to rename the dataframes
    if i == 0:
        indexVals = totalRaceData.index
        labels = totalRaceData['label'].to_list()
        labelsDict = OrderedDict({})
        for key in indexVals:
            for value in labels:
                labelsDict[key] = value
                labels.remove(value)
                break

        labelsDict.update({"geometry": "geometry"})
        labelsDict.move_to_end("geometry", last = False)

        labelsDict.update({"GEOID": "GEOID"})
        labelsDict.move_to_end("GEOID", last = False)

        labelsDict.update({"state": "state"})
        labelsDict.update({"county": "county"})
        labelsDict.update({"tract": "tract"})
        labelsDict.update({"NAME": "NAME"})
    
    # rename the dataframe
    currentDF.columns = currentDF.columns.to_series().map(labelsDict)
    
    # drop the duplciate columns in the dataframe
    currentDF = currentDF.loc[:, ~currentDF.columns.duplicated()].copy()
    
    # only keep the previously selected columns
    currentDF = currentDF[to_keep]
    

    # rename the columns 
    currentDF = currentDF.set_axis(rename, axis = 1)
    
    
    # get the sums for each category and put into a dictonary
    current_sums = currentDF.sum().to_dict()
    current_sums = OrderedDict(current_sums)

    current_sums.update({"County": current})
    current_sums.move_to_end("County", last = False)

    current_sums = dict(current_sums)

    # if first run through
    # get the headers for the final dataframe
    # append to the final dataframe
    if i == 0:
        headers = list(current_sums.keys())
        raceData.append(headers)
        
    # create inner_row to append
    # and append to final dataframe
    inner_row = list(current_sums.values())
    raceData.append(inner_row)
    
    # for periodic checking
    if ((i % 2) == 0):
        print(inner_row)
        
        
        
    # force code to take time
    # do this to avoid a 429 error
    # (too many requests error)
    time.sleep(5)    

['New York', 19020.0, 188400.0, 74668.0, 55648.0, 922033.0, 1631993.0, 888.0, 198833.0, 6178.0, 240993.0]
['Oneida', 1113.0, 3263.0, 6790.0, 5677.0, 196219.0, 229959.0, 70.0, 9304.0, 542.0, 13771.0]
['Ontario', 291.0, 1582.0, 1864.0, 1573.0, 101371.0, 109511.0, 36.0, 1501.0, 388.0, 2769.0]
['Orleans', 38.0, 797.0, 993.0, 955.0, 36345.0, 40904.0, 0.0, 250.0, 136.0, 2383.0]
['Otsego', 259.0, 430.0, 1291.0, 1032.0, 56036.0, 59972.0, 6.0, 814.0, 92.0, 1303.0]
['Queens', 30892.0, 316141.0, 82764.0, 51872.0, 875191.0, 2287388.0, 1078.0, 583459.0, 10326.0, 418429.0]
['Richmond', 2112.0, 17548.0, 10862.0, 8750.0, 352034.0, 474893.0, 219.0, 44246.0, 1026.0, 48958.0]
['Saratoga', 433.0, 1195.0, 4277.0, 3844.0, 211792.0, 228502.0, 41.0, 6697.0, 508.0, 3992.0]
['Schoharie', 72.0, 170.0, 427.0, 355.0, 29791.0, 31222.0, 9.0, 281.0, 48.0, 496.0]
['Seneca', 37.0, 312.0, 543.0, 506.0, 31355.0, 34390.0, 8.0, 285.0, 143.0, 1744.0]
['Steuben', 96.0, 462.0, 1465.0, 1369.0, 91286.0, 96422.0, 3.0, 1480.0, 10

In [58]:
# make into dataframe
raceDF = pd.DataFrame(raceData)

# add header
new_header = raceDF.iloc[0] 
raceDF = raceDF[1:] 
raceDF.columns = new_header 

# view
raceDF

,County,Total: Two or more races: Two races including Some other race,Total: Some other race alone,Total: Two or more races,"Total: Two or more races excluding Some other race, and three or more races",Total: White alone,Total:,Total: Native Hawaiian and Other Pacific Islander alone,Total: Asian alone,Total: American Indian and Alaska Native alone,Total: Black or African American alone
1,Albany,1048.0,4085.0,12787.0,11739.0,231246.0,306968.0,129.0,20192.0,494.0,38035.0
2,Allegany,6.0,216.0,537.0,531.0,44565.0,46688.0,15.0,548.0,107.0,700.0
3,Bronx,23245.0,494363.0,55868.0,32623.0,315533.0,1435068.0,1385.0,52989.0,10116.0,504814.0
4,Broome,535.0,2058.0,6292.0,5757.0,165108.0,193188.0,75.0,8389.0,348.0,10918.0
5,Cattaraugus,144.0,470.0,1581.0,1437.0,70859.0,77121.0,0.0,610.0,2485.0,1116.0
...,...,...,...,...,...,...,...,...,...,...,...
59,Washington,81.0,518.0,869.0,788.0,57686.0,61616.0,7.0,371.0,196.0,1969.0
60,Wayne,188.0,960.0,1915.0,1727.0,84123.0,90519.0,105.0,725.0,148.0,2543.0
61,Westchester,8362.0,110029.0,29968.0,21606.0,622855.0,968890.0,251.0,58140.0,3594.0,144053.0
62,Wyoming,60.0,569.0,815.0,755.0,36882.0,40305.0,0.0,167.0,203.0,1669.0


In [59]:
# write to csv
# write to sex data to csv
# as backup
raceDF.to_csv("NY_raceData.csv")

In [60]:
# check csv saved correctly
# read data in from csv
# this is so you don't have to run through the previous for loop (saves time)

raceDF = pd.read_csv('NY_raceData.csv')
raceDF = raceDF.drop(['Unnamed: 0'], axis = 1)
raceDF

,County,Total: Two or more races: Two races including Some other race,Total: Some other race alone,Total: Two or more races,"Total: Two or more races excluding Some other race, and three or more races",Total: White alone,Total:,Total: Native Hawaiian and Other Pacific Islander alone,Total: Asian alone,Total: American Indian and Alaska Native alone,Total: Black or African American alone
0,Albany,1048.0,4085.0,12787.0,11739.0,231246.0,306968.0,129.0,20192.0,494.0,38035.0
1,Allegany,6.0,216.0,537.0,531.0,44565.0,46688.0,15.0,548.0,107.0,700.0
2,Bronx,23245.0,494363.0,55868.0,32623.0,315533.0,1435068.0,1385.0,52989.0,10116.0,504814.0
3,Broome,535.0,2058.0,6292.0,5757.0,165108.0,193188.0,75.0,8389.0,348.0,10918.0
4,Cattaraugus,144.0,470.0,1581.0,1437.0,70859.0,77121.0,0.0,610.0,2485.0,1116.0
...,...,...,...,...,...,...,...,...,...,...,...
58,Washington,81.0,518.0,869.0,788.0,57686.0,61616.0,7.0,371.0,196.0,1969.0
59,Wayne,188.0,960.0,1915.0,1727.0,84123.0,90519.0,105.0,725.0,148.0,2543.0
60,Westchester,8362.0,110029.0,29968.0,21606.0,622855.0,968890.0,251.0,58140.0,3594.0,144053.0
61,Wyoming,60.0,569.0,815.0,755.0,36882.0,40305.0,0.0,167.0,203.0,1669.0


## Create Dataframe on Education

In [61]:
acs.filter_tables('EDUCATION', by = 'description')

,description,columns
table_name,,
B06009,PLACE OF BIRTH BY EDUCATIONAL ATTAINMENT IN TH...,"[B06009_001E, B06009_002E, B06009_003E, B06009..."
B07009,GEOGRAPHICAL MOBILITY IN THE PAST YEAR BY EDUC...,"[B07009_001E, B07009_002E, B07009_003E, B07009..."
B07409,GEOGRAPHICAL MOBILITY IN THE PAST YEAR BY EDUC...,"[B07409_001E, B07409_002E, B07409_003E, B07409..."
B13014,WOMEN 15 TO 50 YEARS WHO HAD A BIRTH IN THE PA...,"[B13014_001E, B13014_002E, B13014_003E, B13014..."
B14005,SEX BY SCHOOL ENROLLMENT BY EDUCATIONAL ATTAIN...,"[B14005_001E, B14005_002E, B14005_003E, B14005..."
B15001,SEX BY AGE BY EDUCATIONAL ATTAINMENT FOR THE P...,"[B15001_001E, B15001_002E, B15001_003E, B15001..."
B15002,SEX BY EDUCATIONAL ATTAINMENT FOR THE POPULATI...,"[B15002_001E, B15002_002E, B15002_003E, B15002..."
B15003,EDUCATIONAL ATTAINMENT FOR THE POPULATION 25 Y...,"[B15003_001E, B15003_002E, B15003_003E, B15003..."
B16010,EDUCATIONAL ATTAINMENT AND EMPLOYMENT STATUS B...,"[B16010_001E, B16010_002E, B16010_003E, B16010..."


In [74]:
totalEducationData = acs.filter_variables('B15002')
totalEducationData

,label,concept,predicateType,group,limit,predicateOnly,hasGeoCollectionSupport,attributes,required
B15002_014E,Estimate!!Total:!!Male:!!Associate's degree,SEX BY EDUCATIONAL ATTAINMENT FOR THE POPULATI...,int,B15002,0,NaN,NaN,"B15002_014EA,B15002_014M,B15002_014MA",NaN
B15002_013E,"Estimate!!Total:!!Male:!!Some college, 1 or mo...",SEX BY EDUCATIONAL ATTAINMENT FOR THE POPULATI...,int,B15002,0,NaN,NaN,"B15002_013EA,B15002_013M,B15002_013MA",NaN
B15002_016E,Estimate!!Total:!!Male:!!Master's degree,SEX BY EDUCATIONAL ATTAINMENT FOR THE POPULATI...,int,B15002,0,NaN,NaN,"B15002_016EA,B15002_016M,B15002_016MA",NaN
B15002_015E,Estimate!!Total:!!Male:!!Bachelor's degree,SEX BY EDUCATIONAL ATTAINMENT FOR THE POPULATI...,int,B15002,0,NaN,NaN,"B15002_015EA,B15002_015M,B15002_015MA",NaN
B15002_018E,Estimate!!Total:!!Male:!!Doctorate degree,SEX BY EDUCATIONAL ATTAINMENT FOR THE POPULATI...,int,B15002,0,NaN,NaN,"B15002_018EA,B15002_018M,B15002_018MA",NaN
B15002_017E,Estimate!!Total:!!Male:!!Professional school d...,SEX BY EDUCATIONAL ATTAINMENT FOR THE POPULATI...,int,B15002,0,NaN,NaN,"B15002_017EA,B15002_017M,B15002_017MA",NaN
B15002_019E,Estimate!!Total:!!Female:,SEX BY EDUCATIONAL ATTAINMENT FOR THE POPULATI...,int,B15002,0,NaN,NaN,"B15002_019EA,B15002_019M,B15002_019MA",NaN
B15002_010E,"Estimate!!Total:!!Male:!!12th grade, no diploma",SEX BY EDUCATIONAL ATTAINMENT FOR THE POPULATI...,int,B15002,0,NaN,NaN,"B15002_010EA,B15002_010M,B15002_010MA",NaN
B15002_012E,"Estimate!!Total:!!Male:!!Some college, less th...",SEX BY EDUCATIONAL ATTAINMENT FOR THE POPULATI...,int,B15002,0,NaN,NaN,"B15002_012EA,B15002_012M,B15002_012MA",NaN
B15002_011E,Estimate!!Total:!!Male:!!High school graduate ...,SEX BY EDUCATIONAL ATTAINMENT FOR THE POPULATI...,int,B15002,0,NaN,NaN,"B15002_011EA,B15002_011M,B15002_011MA",NaN


In [70]:
# educationData = []

# the columns we are looking for
to_keep = ["Estimate!!Total:!!Male:", "Estimate!!Total:!!Male:!!Associate's degree", 
           "Estimate!!Total:!!Male:!!Some college, 1 or more years, no degree",
           "Estimate!!Total:!!Male:!!Master's degree",
           "Estimate!!Total:!!Male:!!Bachelor's degree",
           "Estimate!!Total:!!Male:!!Doctorate degree",
           "Estimate!!Total:!!Male:!!Professional school degree",
           "Estimate!!Total:!!Male:!!12th grade, no diploma",
           "Estimate!!Total:!!Male:!!High school graduate (includes equivalency)",
           "Estimate!!Total:!!Female:",
           "Estimate!!Total:!!Female:!!Associate's degree", 
           "Estimate!!Total:!!Female:!!Some college, 1 or more years, no degree",
           "Estimate!!Total:!!Female:!!Master's degree",
           "Estimate!!Total:!!Female:!!Bachelor's degree",
           "Estimate!!Total:!!Female:!!Doctorate degree",
           "Estimate!!Total:!!Female:!!Professional school degree",
           "Estimate!!Total:!!Female:!!12th grade, no diploma",
           "Estimate!!Total:!!Female:!!High school graduate (includes equivalency)"]

# renamed column names
rename = ["Total: Male:",
           "Total: Male: Associate's degree", 
           "Total: Male: Some college, 1 or more years, no degree",
           "Total: Male: Master's degree",
           "Total: Male: Bachelor's degree",
           "Total: Male: Doctorate degree",
           "Total: Male: Professional school degree",
           "Total: Male: 12th grade, no diploma",
           "Total: Male: High school graduate (includes equivalency)",
           "Total: Female:",
           "Total: Female: Associate's degree", 
           "Total: Female: Some college, 1 or more years, no degree",
           "Total: Female: Master's degree",
           "Total: Female: Bachelor's degree",
           "Total: Female: Doctorate degree",
           "Total: Female: Professional school degree",
           "Total: Female: 12th grade, no diploma",
           "Total: Female: High school graduate (includes equivalency)"]

# run in groups of 10 because of 'too many requests' error

for i in range(10, 63, 1):

    name = counties[i] + ", NY"
    currentDF = acs.from_county(name, variables = 'B15002')
    current = counties[i]
    
    # if first round through
    # create labels to rename the dataframes
    if i == 0:
        indexVals = totalEducationData.index
        labels = totalEducationData['label'].to_list()
        labelsDict = OrderedDict({})
        for key in indexVals:
            for value in labels:
                labelsDict[key] = value
                labels.remove(value)
                break

        labelsDict.update({"geometry": "geometry"})
        labelsDict.move_to_end("geometry", last = False)

        labelsDict.update({"GEOID": "GEOID"})
        labelsDict.move_to_end("GEOID", last = False)

        labelsDict.update({"state": "state"})
        labelsDict.update({"county": "county"})
        labelsDict.update({"tract": "tract"})
        labelsDict.update({"NAME": "NAME"})
    
    # rename the dataframe
    currentDF.columns = currentDF.columns.to_series().map(labelsDict)
    
    # drop the duplciate columns in the county dataframe
    currentDF = currentDF.loc[:, ~currentDF.columns.duplicated()].copy()
    
    # only keep the previously selected columns
    currentDF = currentDF[to_keep]
    
    # rename the columns 
    currentDF = currentDF.set_axis(rename, axis = 1)
    
    
    # get the sums for each category and put into a dictonary
    current_sums = currentDF.sum().to_dict()
    current_sums = OrderedDict(current_sums)

    current_sums.update({"County": current})
    current_sums.move_to_end("County", last = False)

    current_sums = dict(current_sums)

    # if first run through
    # get the headers for the final dataframe
    # append to the final dataframe
    if i == 0:
        headers = list(current_sums.keys())
        educationData.append(headers)
        
    # create inner_row to append
    # and append to final dataframe
    inner_row = list(current_sums.values())
    educationData.append(inner_row)
    

    # for periodic checking
    if ((i % 2) == 0):
        print(inner_row)
        
        
        
    # force code to take time
    # do this to avoid a 429 error
    # (too many requests error)
    time.sleep(5)    

    

['Columbia', 22346.0, 2181.0, 2262.0, 2076.0, 3695.0, 390.0, 716.0, 374.0, 7255.0, 23228.0, 3124.0, 2586.0, 3074.0, 4146.0, 258.0, 534.0, 222.0, 6398.0]
['Delaware', 16278.0, 1848.0, 1677.0, 1043.0, 1979.0, 109.0, 145.0, 401.0, 6380.0, 16509.0, 2064.0, 1927.0, 1678.0, 2162.0, 132.0, 162.0, 197.0, 5939.0]
['Erie', 306207.0, 32590.0, 38779.0, 27185.0, 62120.0, 5653.0, 8728.0, 5452.0, 87502.0, 339774.0, 44140.0, 41734.0, 44805.0, 63278.0, 4239.0, 5923.0, 4750.0, 90700.0]
['Franklin', 18987.0, 1930.0, 2085.0, 792.0, 1797.0, 80.0, 296.0, 277.0, 8174.0, 16078.0, 2470.0, 1959.0, 1807.0, 1663.0, 107.0, 147.0, 231.0, 5040.0]
['Genesee', 20676.0, 2333.0, 2640.0, 1120.0, 2560.0, 190.0, 276.0, 364.0, 8135.0, 20658.0, 3696.0, 2339.0, 1812.0, 2947.0, 104.0, 201.0, 165.0, 6729.0]
['Hamilton', 1765.0, 194.0, 221.0, 122.0, 159.0, 13.0, 22.0, 136.0, 492.0, 1802.0, 192.0, 451.0, 101.0, 165.0, 8.0, 11.0, 89.0, 477.0]
['Jefferson', 36594.0, 3935.0, 5310.0, 1633.0, 4992.0, 187.0, 443.0, 565.0, 13555.0, 3494

In [71]:
# make into dataframe
educationDF = pd.DataFrame(educationData)

# add header
new_header = educationDF.iloc[0] 
educationDF = educationDF[1:] 
educationDF.columns = new_header 

# view
educationDF

,County,Total: Male:,Total: Male: Associate's degree,"Total: Male: Some college, 1 or more years, no degree",Total: Male: Master's degree,Total: Male: Bachelor's degree,Total: Male: Doctorate degree,Total: Male: Professional school degree,"Total: Male: 12th grade, no diploma",Total: Male: High school graduate (includes equivalency),Total: Female:,Total: Female: Associate's degree,"Total: Female: Some college, 1 or more years, no degree",Total: Female: Master's degree,Total: Female: Bachelor's degree,Total: Female: Doctorate degree,Total: Female: Professional school degree,"Total: Female: 12th grade, no diploma",Total: Female: High school graduate (includes equivalency)
1,Albany,97391.0,10565.0,10655.0,11689.0,22150.0,3099.0,4166.0,1670.0,22090.0,107527.0,13526.0,10639.0,17281.0,21745.0,2422.0,3150.0,1303.0,24952.0
2,Allegany,14874.0,1908.0,1755.0,968.0,1448.0,275.0,115.0,187.0,5813.0,15164.0,2278.0,1982.0,1500.0,1741.0,125.0,96.0,187.0,5199.0
3,Bronx,419713.0,27221.0,54467.0,17658.0,49966.0,3124.0,4679.0,15514.0,129378.0,509833.0,41247.0,67166.0,32755.0,70055.0,3097.0,5216.0,18139.0,130194.0
4,Broome,61426.0,7241.0,7175.0,5135.0,10237.0,1054.0,1410.0,1153.0,19412.0,66638.0,9380.0,7349.0,7291.0,9544.0,845.0,829.0,964.0,20890.0
5,Cattaraugus,25857.0,2800.0,2824.0,1357.0,2600.0,242.0,207.0,560.0,10919.0,26992.0,4077.0,3013.0,2552.0,2828.0,87.0,208.0,520.0,9796.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,Washington,22914.0,1956.0,2221.0,1131.0,2426.0,134.0,292.0,342.0,9746.0,22078.0,3062.0,2666.0,2058.0,2855.0,138.0,201.0,237.0,7683.0
60,Wayne,31367.0,3304.0,3930.0,1807.0,4097.0,213.0,272.0,771.0,11781.0,32547.0,5482.0,3945.0,3262.0,4264.0,193.0,359.0,474.0,9750.0
61,Westchester,315674.0,18899.0,32071.0,44240.0,75042.0,8179.0,22392.0,7350.0,61529.0,352616.0,23798.0,33727.0,68238.0,86954.0,6546.0,15342.0,7167.0,66963.0
62,Wyoming,15680.0,1721.0,1763.0,615.0,1551.0,49.0,46.0,317.0,6913.0,13353.0,2339.0,1574.0,1035.0,1672.0,17.0,146.0,164.0,4701.0


In [72]:
# write to csv
# write to sex data to csv
# as backup
educationDF.to_csv("NY_educationData.csv")

In [73]:
# check csv saved correctly
# read data in from csv
# this is so you don't have to run through the previous for loop (saves time)

educationDF = pd.read_csv('NY_educationData.csv')
educationDF = educationDF.drop(['Unnamed: 0'], axis = 1)
educationDF

,County,Total: Male:,Total: Male: Associate's degree,"Total: Male: Some college, 1 or more years, no degree",Total: Male: Master's degree,Total: Male: Bachelor's degree,Total: Male: Doctorate degree,Total: Male: Professional school degree,"Total: Male: 12th grade, no diploma",Total: Male: High school graduate (includes equivalency),Total: Female:,Total: Female: Associate's degree,"Total: Female: Some college, 1 or more years, no degree",Total: Female: Master's degree,Total: Female: Bachelor's degree,Total: Female: Doctorate degree,Total: Female: Professional school degree,"Total: Female: 12th grade, no diploma",Total: Female: High school graduate (includes equivalency)
0,Albany,97391.0,10565.0,10655.0,11689.0,22150.0,3099.0,4166.0,1670.0,22090.0,107527.0,13526.0,10639.0,17281.0,21745.0,2422.0,3150.0,1303.0,24952.0
1,Allegany,14874.0,1908.0,1755.0,968.0,1448.0,275.0,115.0,187.0,5813.0,15164.0,2278.0,1982.0,1500.0,1741.0,125.0,96.0,187.0,5199.0
2,Bronx,419713.0,27221.0,54467.0,17658.0,49966.0,3124.0,4679.0,15514.0,129378.0,509833.0,41247.0,67166.0,32755.0,70055.0,3097.0,5216.0,18139.0,130194.0
3,Broome,61426.0,7241.0,7175.0,5135.0,10237.0,1054.0,1410.0,1153.0,19412.0,66638.0,9380.0,7349.0,7291.0,9544.0,845.0,829.0,964.0,20890.0
4,Cattaraugus,25857.0,2800.0,2824.0,1357.0,2600.0,242.0,207.0,560.0,10919.0,26992.0,4077.0,3013.0,2552.0,2828.0,87.0,208.0,520.0,9796.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,Washington,22914.0,1956.0,2221.0,1131.0,2426.0,134.0,292.0,342.0,9746.0,22078.0,3062.0,2666.0,2058.0,2855.0,138.0,201.0,237.0,7683.0
59,Wayne,31367.0,3304.0,3930.0,1807.0,4097.0,213.0,272.0,771.0,11781.0,32547.0,5482.0,3945.0,3262.0,4264.0,193.0,359.0,474.0,9750.0
60,Westchester,315674.0,18899.0,32071.0,44240.0,75042.0,8179.0,22392.0,7350.0,61529.0,352616.0,23798.0,33727.0,68238.0,86954.0,6546.0,15342.0,7167.0,66963.0
61,Wyoming,15680.0,1721.0,1763.0,615.0,1551.0,49.0,46.0,317.0,6913.0,13353.0,2339.0,1574.0,1035.0,1672.0,17.0,146.0,164.0,4701.0


## Create Dataframe on Health Insurance

In [76]:
acs.filter_tables('HEALTH INSURANCE', by = 'description')

,description,columns
table_name,,
B18135,AGE BY DISABILITY STATUS BY HEALTH INSURANCE C...,"[B18135_001E, B18135_002E, B18135_003E, B18135..."
B27001,HEALTH INSURANCE COVERAGE STATUS BY SEX BY AGE,"[B27001_001E, B27001_002E, B27001_003E, B27001..."
B27002,PRIVATE HEALTH INSURANCE STATUS BY SEX BY AGE,"[B27002_001E, B27002_002E, B27002_003E, B27002..."
B27003,PUBLIC HEALTH INSURANCE STATUS BY SEX BY AGE,"[B27003_001E, B27003_002E, B27003_003E, B27003..."
B27010,TYPES OF HEALTH INSURANCE COVERAGE BY AGE,"[B27010_001E, B27010_002E, B27010_003E, B27010..."
B27011,HEALTH INSURANCE COVERAGE STATUS AND TYPE BY E...,"[B27011_001E, B27011_002E, B27011_003E, B27011..."
B27015,HEALTH INSURANCE COVERAGE STATUS AND TYPE BY H...,"[B27015_001E, B27015_002E, B27015_003E, B27015..."
B27019,HEALTH INSURANCE COVERAGE STATUS AND TYPE BY A...,"[B27019_001E, B27019_002E, B27019_003E, B27019..."
B27020,HEALTH INSURANCE COVERAGE STATUS AND TYPE BY C...,"[B27020_001E, B27020_002E, B27020_003E, B27020..."


In [80]:
totalHealthData = acs.filter_variables('B27001')
totalHealthData

,label,concept,predicateType,group,limit,predicateOnly,hasGeoCollectionSupport,attributes,required
B27001_031E,Estimate!!Total:!!Female:!!Under 6 years:,HEALTH INSURANCE COVERAGE STATUS BY SEX BY AGE,int,B27001,0,NaN,NaN,"B27001_031EA,B27001_031M,B27001_031MA",NaN
B27001_032E,Estimate!!Total:!!Female:!!Under 6 years:!!Wit...,HEALTH INSURANCE COVERAGE STATUS BY SEX BY AGE,int,B27001,0,NaN,NaN,"B27001_032EA,B27001_032M,B27001_032MA",NaN
B27001_030E,Estimate!!Total:!!Female:,HEALTH INSURANCE COVERAGE STATUS BY SEX BY AGE,int,B27001,0,NaN,NaN,"B27001_030EA,B27001_030M,B27001_030MA",NaN
B27001_023E,Estimate!!Total:!!Male:!!55 to 64 years:!!No h...,HEALTH INSURANCE COVERAGE STATUS BY SEX BY AGE,int,B27001,0,NaN,NaN,"B27001_023EA,B27001_023M,B27001_023MA",NaN
B27001_024E,Estimate!!Total:!!Male:!!65 to 74 years:,HEALTH INSURANCE COVERAGE STATUS BY SEX BY AGE,int,B27001,0,NaN,NaN,"B27001_024EA,B27001_024M,B27001_024MA",NaN
B27001_021E,Estimate!!Total:!!Male:!!55 to 64 years:,HEALTH INSURANCE COVERAGE STATUS BY SEX BY AGE,int,B27001,0,NaN,NaN,"B27001_021EA,B27001_021M,B27001_021MA",NaN
B27001_022E,Estimate!!Total:!!Male:!!55 to 64 years:!!With...,HEALTH INSURANCE COVERAGE STATUS BY SEX BY AGE,int,B27001,0,NaN,NaN,"B27001_022EA,B27001_022M,B27001_022MA",NaN
B27001_027E,Estimate!!Total:!!Male:!!75 years and over:,HEALTH INSURANCE COVERAGE STATUS BY SEX BY AGE,int,B27001,0,NaN,NaN,"B27001_027EA,B27001_027M,B27001_027MA",NaN
B27001_028E,Estimate!!Total:!!Male:!!75 years and over:!!W...,HEALTH INSURANCE COVERAGE STATUS BY SEX BY AGE,int,B27001,0,NaN,NaN,"B27001_028EA,B27001_028M,B27001_028MA",NaN
B27001_025E,Estimate!!Total:!!Male:!!65 to 74 years:!!With...,HEALTH INSURANCE COVERAGE STATUS BY SEX BY AGE,int,B27001,0,NaN,NaN,"B27001_025EA,B27001_025M,B27001_025MA",NaN


In [103]:
healthData = []

# the columns we need to drop
to_drop = ['GEOID', 'geometry',
           'Estimate!!Total:!!Female:!!Under 6 years:',
           'Estimate!!Total:!!Female:!!Under 6 years:!!With health insurance coverage',
           'Estimate!!Total:!!Female:!!6 to 18 years:!!With health insurance coverage',
           'Estimate!!Total:!!Female:!!6 to 18 years:!!No health insurance coverage',
           'Estimate!!Total:!!Female:!!Under 6 years:!!No health insurance coverage',
           'Estimate!!Total:!!Male:!!Under 6 years:',
           'Estimate!!Total:!!Male:!!Under 6 years:!!With health insurance coverage',
           'Estimate!!Total:!!Male:!!6 to 18 years:!!With health insurance coverage',
           'Estimate!!Total:!!Male:!!6 to 18 years:!!No health insurance coverage',
           'Estimate!!Total:!!Male:!!Under 6 years:!!No health insurance coverage',
           'Estimate!!Total:!!Female:!!6 to 18 years:',
           'Estimate!!Total:!!Male:!!6 to 18 years:',
           'state', 'county', 'tract', "NAME"]


# renamed column names
rename = ['Total:',
          'Total: Male:',
          'Total: Male: 19 to 25 years:',
          'Total: Male: 19 to 25 years: With health insurance coverage',
          'Total: Male: 19 to 25 years: No health insurance coverage',
          'Total: Male: 26 to 34 years:',
          'Total: Male: 26 to 34 years: With health insurance coverage',
          'Total: Male: 26 to 34 years: No health insurance coverage',
          'Total: Male: 35 to 44 years:',
          'Total: Male: 35 to 44 years: With health insurance coverage',
          'Total: Male: 35 to 44 years: No health insurance coverage',
          'Total: Male: 45 to 54 years:',
          'Total: Male: 45 to 54 years: With health insurance coverage',
          'Total: Male: 45 to 54 years: No health insurance coverage',
          'Total: Male: 55 to 64 years:',
          'Total: Male: 55 to 64 years: With health insurance coverage',
          'Total: Male: 55 to 64 years: No health insurance coverage',
          'Total: Male: 65 to 74 years:',
          'Total: Male: 65 to 74 years: With health insurance coverage',
          'Total: Male: 65 to 74 years: No health insurance coverage',
          'Total: Male: 75 years and over:',
          'Total: Male: 75 years and over: With health insurance coverage',
          'Total: Male: 75 years and over: No health insurance coverage',
          'Total: Female:',
          'Total: Female: 19 to 25 years:',
          'Total: Female: 19 to 25 years: With health insurance coverage',
          'Total: Female: 19 to 25 years: No health insurance coverage',
          'Total: Female: 26 to 34 years:',
          'Total: Female: 26 to 34 years: With health insurance coverage',
          'Total: Female: 26 to 34 years: No health insurance coverage',
          'Total: Female: 35 to 44 years:',
          'Total: Female: 35 to 44 years: With health insurance coverage',
          'Total: Female: 35 to 44 years: No health insurance coverage',
          'Total: Female: 45 to 54 years:',
          'Total: Female: 45 to 54 years: With health insurance coverage',
          'Total: Female: 45 to 54 years: No health insurance coverage',
          'Total: Female: 55 to 64 years:',
          'Total: Female: 55 to 64 years: With health insurance coverage',
          'Total: Female: 55 to 64 years: No health insurance coverage',
          'Total: Female: 65 to 74 years:',
          'Total: Female: 65 to 74 years: With health insurance coverage',
          'Total: Female: 65 to 74 years: No health insurance coverage',
          'Total: Female: 75 years and over:',
          'Total: Female: 75 years and over: With health insurance coverage',
          'Total: Female: 75 years and over: No health insurance coverage']

# run in groups of 10 because of 'too many requests' error

for i in range(len(counties)):

    name = counties[i] + ", NY"
    currentDF = acs.from_county(name, variables = 'B27001')
    current = counties[i]
    
    # if first round through
    # create labels to rename the dataframes
    if i == 0:
        indexVals = totalHealthData.index
        labels = totalHealthData['label'].to_list()
        labelsDict = OrderedDict({})
        for key in indexVals:
            for value in labels:
                labelsDict[key] = value
                labels.remove(value)
                break

        labelsDict.update({"geometry": "geometry"})
        labelsDict.move_to_end("geometry", last = False)

        labelsDict.update({"GEOID": "GEOID"})
        labelsDict.move_to_end("GEOID", last = False)

        labelsDict.update({"state": "state"})
        labelsDict.update({"county": "county"})
        labelsDict.update({"tract": "tract"})
        labelsDict.update({"NAME": "NAME"})
        
    
    # rename the dataframe
    currentDF.columns = currentDF.columns.to_series().map(labelsDict)
    
    # drop select columns
    currentDF = currentDF.drop(to_drop, axis = 1)

    # drop the duplciate columns in the county dataframe
    currentDF = currentDF.loc[:, ~currentDF.columns.duplicated()].copy()
    
    # rename the columns 
    currentDF = currentDF.set_axis(rename, axis = 1)
    
    
    
    # get the sums for each category and put into a dictonary
    current_sums = currentDF.sum().to_dict()
    current_sums = OrderedDict(current_sums)

    current_sums.update({"County": current})
    current_sums.move_to_end("County", last = False)

    current_sums = dict(current_sums)

    # if first run through
    # get the headers for the final dataframe
    # append to the final dataframe
    if i == 0:
        headers = list(current_sums.keys())
        healthData.append(headers)
        
    # create inner_row to append
    # and append to final dataframe
    inner_row = list(current_sums.values())
    healthData.append(inner_row)
    

    # for periodic checking
    if ((i % 2) == 0):
        print(inner_row)
        
        
        
    # force code to take time
    # do this to avoid a 429 error
    # (too many requests error)
    time.sleep(5)    

    

['Albany', 303159.0, 146903.0, 20660.0, 19528.0, 1132.0, 17899.0, 16040.0, 1859.0, 17006.0, 15983.0, 1023.0, 18696.0, 17828.0, 868.0, 19453.0, 18883.0, 570.0, 13162.0, 13110.0, 52.0, 7795.0, 7774.0, 21.0, 156256.0, 22162.0, 21451.0, 711.0, 17547.0, 16583.0, 964.0, 17741.0, 17173.0, 568.0, 19608.0, 19072.0, 536.0, 20960.0, 20615.0, 345.0, 15284.0, 15238.0, 46.0, 12072.0, 12049.0, 23.0]
['Bronx', 1414023.0, 663121.0, 76306.0, 64971.0, 11335.0, 95653.0, 73078.0, 22575.0, 81736.0, 65861.0, 15875.0, 82036.0, 71229.0, 10807.0, 69513.0, 63507.0, 6006.0, 40844.0, 39612.0, 1232.0, 25928.0, 25738.0, 190.0, 750902.0, 75738.0, 69176.0, 6562.0, 105289.0, 92554.0, 12735.0, 97223.0, 87290.0, 9933.0, 97957.0, 89155.0, 8802.0, 86057.0, 81273.0, 4784.0, 56457.0, 55422.0, 1035.0, 47046.0, 46424.0, 622.0]
['Cattaraugus', 76342.0, 37893.0, 3355.0, 2783.0, 572.0, 3815.0, 3288.0, 527.0, 4091.0, 3700.0, 391.0, 5028.0, 4678.0, 350.0, 5774.0, 5449.0, 325.0, 4002.0, 3978.0, 24.0, 2473.0, 2462.0, 11.0, 38449.0, 3

In [106]:
# make into dataframe
healthDF = pd.DataFrame(healthData)

# add header
new_header = healthDF.iloc[0] 
healthDF = healthDF[1:] 
healthDF.columns = new_header 

# view
healthDF

,County,Total:,Total: Male:,Total: Male: 19 to 25 years:,Total: Male: 19 to 25 years: With health insurance coverage,Total: Male: 19 to 25 years: No health insurance coverage,Total: Male: 26 to 34 years:,Total: Male: 26 to 34 years: With health insurance coverage,Total: Male: 26 to 34 years: No health insurance coverage,Total: Male: 35 to 44 years:,...,Total: Female: 45 to 54 years: No health insurance coverage,Total: Female: 55 to 64 years:,Total: Female: 55 to 64 years: With health insurance coverage,Total: Female: 55 to 64 years: No health insurance coverage,Total: Female: 65 to 74 years:,Total: Female: 65 to 74 years: With health insurance coverage,Total: Female: 65 to 74 years: No health insurance coverage,Total: Female: 75 years and over:,Total: Female: 75 years and over: With health insurance coverage,Total: Female: 75 years and over: No health insurance coverage
1,Albany,303159.0,146903.0,20660.0,19528.0,1132.0,17899.0,16040.0,1859.0,17006.0,...,536.0,20960.0,20615.0,345.0,15284.0,15238.0,46.0,12072.0,12049.0,23.0
2,Allegany,46221.0,23452.0,3223.0,2965.0,258.0,2146.0,1871.0,275.0,2281.0,...,167.0,3267.0,3129.0,138.0,2473.0,2469.0,4.0,1857.0,1857.0,0.0
3,Bronx,1414023.0,663121.0,76306.0,64971.0,11335.0,95653.0,73078.0,22575.0,81736.0,...,8802.0,86057.0,81273.0,4784.0,56457.0,55422.0,1035.0,47046.0,46424.0,622.0
4,Broome,190442.0,93663.0,12962.0,11717.0,1245.0,9612.0,8318.0,1294.0,9572.0,...,393.0,13701.0,13237.0,464.0,9906.0,9906.0,0.0,9741.0,9704.0,37.0
5,Cattaraugus,76342.0,37893.0,3355.0,2783.0,572.0,3815.0,3288.0,527.0,4091.0,...,276.0,5799.0,5574.0,225.0,4183.0,4166.0,17.0,3350.0,3334.0,16.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,Washington,58570.0,29155.0,2249.0,2041.0,208.0,3015.0,2408.0,607.0,3501.0,...,164.0,4642.0,4510.0,132.0,3425.0,3413.0,12.0,2569.0,2566.0,3.0
60,Wayne,89743.0,44503.0,3637.0,3341.0,296.0,4598.0,4099.0,499.0,4992.0,...,270.0,7053.0,6930.0,123.0,5043.0,5019.0,24.0,3591.0,3584.0,7.0
61,Westchester,956650.0,462558.0,40497.0,36550.0,3947.0,50019.0,41008.0,9011.0,59614.0,...,3824.0,67718.0,65232.0,2486.0,46598.0,46179.0,419.0,43472.0,43265.0,207.0
62,Wyoming,37846.0,19123.0,1374.0,1188.0,186.0,2346.0,2100.0,246.0,2149.0,...,117.0,2832.0,2779.0,53.0,2080.0,2076.0,4.0,1510.0,1510.0,0.0


In [107]:
# write to csv
# write to sex data to csv
# as backup
healthDF.to_csv("NY_healthData.csv")

In [108]:
# check csv saved correctly
# read data in from csv
# this is so you don't have to run through the previous for loop (saves time)

healthDF = pd.read_csv('NY_healthData.csv')
healthDF = healthDF.drop(['Unnamed: 0'], axis = 1)
healthDF

,County,Total:,Total: Male:,Total: Male: 19 to 25 years:,Total: Male: 19 to 25 years: With health insurance coverage,Total: Male: 19 to 25 years: No health insurance coverage,Total: Male: 26 to 34 years:,Total: Male: 26 to 34 years: With health insurance coverage,Total: Male: 26 to 34 years: No health insurance coverage,Total: Male: 35 to 44 years:,...,Total: Female: 45 to 54 years: No health insurance coverage,Total: Female: 55 to 64 years:,Total: Female: 55 to 64 years: With health insurance coverage,Total: Female: 55 to 64 years: No health insurance coverage,Total: Female: 65 to 74 years:,Total: Female: 65 to 74 years: With health insurance coverage,Total: Female: 65 to 74 years: No health insurance coverage,Total: Female: 75 years and over:,Total: Female: 75 years and over: With health insurance coverage,Total: Female: 75 years and over: No health insurance coverage
0,Albany,303159.0,146903.0,20660.0,19528.0,1132.0,17899.0,16040.0,1859.0,17006.0,...,536.0,20960.0,20615.0,345.0,15284.0,15238.0,46.0,12072.0,12049.0,23.0
1,Allegany,46221.0,23452.0,3223.0,2965.0,258.0,2146.0,1871.0,275.0,2281.0,...,167.0,3267.0,3129.0,138.0,2473.0,2469.0,4.0,1857.0,1857.0,0.0
2,Bronx,1414023.0,663121.0,76306.0,64971.0,11335.0,95653.0,73078.0,22575.0,81736.0,...,8802.0,86057.0,81273.0,4784.0,56457.0,55422.0,1035.0,47046.0,46424.0,622.0
3,Broome,190442.0,93663.0,12962.0,11717.0,1245.0,9612.0,8318.0,1294.0,9572.0,...,393.0,13701.0,13237.0,464.0,9906.0,9906.0,0.0,9741.0,9704.0,37.0
4,Cattaraugus,76342.0,37893.0,3355.0,2783.0,572.0,3815.0,3288.0,527.0,4091.0,...,276.0,5799.0,5574.0,225.0,4183.0,4166.0,17.0,3350.0,3334.0,16.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,Washington,58570.0,29155.0,2249.0,2041.0,208.0,3015.0,2408.0,607.0,3501.0,...,164.0,4642.0,4510.0,132.0,3425.0,3413.0,12.0,2569.0,2566.0,3.0
59,Wayne,89743.0,44503.0,3637.0,3341.0,296.0,4598.0,4099.0,499.0,4992.0,...,270.0,7053.0,6930.0,123.0,5043.0,5019.0,24.0,3591.0,3584.0,7.0
60,Westchester,956650.0,462558.0,40497.0,36550.0,3947.0,50019.0,41008.0,9011.0,59614.0,...,3824.0,67718.0,65232.0,2486.0,46598.0,46179.0,419.0,43472.0,43265.0,207.0
61,Wyoming,37846.0,19123.0,1374.0,1188.0,186.0,2346.0,2100.0,246.0,2149.0,...,117.0,2832.0,2779.0,53.0,2080.0,2076.0,4.0,1510.0,1510.0,0.0


## Create Dataframe on Income

In [112]:
acs.filter_tables('MEDIAN INCOME', by = 'description')

,description,columns
table_name,,
B06011,MEDIAN INCOME IN THE PAST 12 MONTHS (IN 2019 I...,"[B06011_001E, B06011_002E, B06011_003E, B06011..."
B07011,MEDIAN INCOME IN THE PAST 12 MONTHS (IN 2019 I...,"[B07011_001E, B07011_002E, B07011_003E, B07011..."
B07411,MEDIAN INCOME IN THE PAST 12 MONTHS (IN 2019 I...,"[B07411_001E, B07411_002E, B07411_003E, B07411..."
B19326,MEDIAN INCOME IN THE PAST 12 MONTHS (IN 2019 I...,"[B19326_001E, B19326_002E, B19326_003E, B19326..."
B21004,MEDIAN INCOME IN THE PAST 12 MONTHS (IN 2019 I...,"[B21004_001E, B21004_002E, B21004_003E, B21004..."


In [118]:
totalIncomeData = acs.filter_variables('B19326')
totalIncomeData

,label,concept,predicateType,group,limit,predicateOnly,hasGeoCollectionSupport,attributes,required
B19326_001E,Estimate!!Median income in the past 12 months ...,MEDIAN INCOME IN THE PAST 12 MONTHS (IN 2019 I...,int,B19326,0,NaN,NaN,"B19326_001EA,B19326_001M,B19326_001MA",NaN
B19326_002E,Estimate!!Median income in the past 12 months ...,MEDIAN INCOME IN THE PAST 12 MONTHS (IN 2019 I...,int,B19326,0,NaN,NaN,"B19326_002EA,B19326_002M,B19326_002MA",NaN
B19326_003E,Estimate!!Median income in the past 12 months ...,MEDIAN INCOME IN THE PAST 12 MONTHS (IN 2019 I...,int,B19326,0,NaN,NaN,"B19326_003EA,B19326_003M,B19326_003MA",NaN
B19326_004E,Estimate!!Median income in the past 12 months ...,MEDIAN INCOME IN THE PAST 12 MONTHS (IN 2019 I...,int,B19326,0,NaN,NaN,"B19326_004EA,B19326_004M,B19326_004MA",NaN
B19326_005E,Estimate!!Median income in the past 12 months ...,MEDIAN INCOME IN THE PAST 12 MONTHS (IN 2019 I...,int,B19326,0,NaN,NaN,"B19326_005EA,B19326_005M,B19326_005MA",NaN
B19326_006E,Estimate!!Median income in the past 12 months ...,MEDIAN INCOME IN THE PAST 12 MONTHS (IN 2019 I...,int,B19326,0,NaN,NaN,"B19326_006EA,B19326_006M,B19326_006MA",NaN
B19326_007E,Estimate!!Median income in the past 12 months ...,MEDIAN INCOME IN THE PAST 12 MONTHS (IN 2019 I...,int,B19326,0,NaN,NaN,"B19326_007EA,B19326_007M,B19326_007MA",NaN


In [123]:
incomeData = []

# columns to drop
to_drop = ['GEOID', 'geometry', 'state', 'county', 'tract', "NAME"]

# renamed column names
rename = ['Total (dollars):',
          'Total Male (dollars):',
          'Total Male: Full-Time, Year-Round (dollars)',
          'Total Male: Other (dollars)',
          'Total Female (dollars):',
          'Total Female: Full-Time, Year-Round (dollars)',
          'Total Female: Other (dollars)']


for i in range(len(counties)):

    name = counties[i] + ", NY"
    currentDF = acs.from_county(name, variables = 'B19326')
    current = counties[i]
    
    # if first round through
    # create labels to rename the dataframes
    if i == 0:
        indexVals = totalIncomeData.index
        labels = totalIncomeData['label'].to_list()
        labelsDict = OrderedDict({})
        for key in indexVals:
            for value in labels:
                labelsDict[key] = value
                labels.remove(value)
                break

        labelsDict.update({"geometry": "geometry"})
        labelsDict.move_to_end("geometry", last = False)

        labelsDict.update({"GEOID": "GEOID"})
        labelsDict.move_to_end("GEOID", last = False)

        labelsDict.update({"state": "state"})
        labelsDict.update({"county": "county"})
        labelsDict.update({"tract": "tract"})
        labelsDict.update({"NAME": "NAME"})
        
    
    # rename the dataframe
    currentDF.columns = currentDF.columns.to_series().map(labelsDict)
    
    # drop columns
    currentDF = currentDF.drop(to_drop, axis = 1)
    
    # rename the columns 
    currentDF = currentDF.set_axis(rename, axis = 1)
    
    
    # get the sums for each category and put into a dictonary
    current_sums = currentDF.sum().to_dict()
    current_sums = OrderedDict(current_sums)

    current_sums.update({"County": current})
    current_sums.move_to_end("County", last = False)

    current_sums = dict(current_sums)

    # if first run through
    # get the headers for the final dataframe
    # append to the final dataframe
    if i == 0:
        headers = list(current_sums.keys())
        incomeData.append(headers)
        
    # create inner_row to append
    # and append to final dataframe
    inner_row = list(current_sums.values())
    incomeData.append(inner_row)
    

    # for periodic checking
    if ((i % 2) == 0):
        print(inner_row)
        
        
        
    # force code to take time
    # do this to avoid a 429 error
    # (too many requests error)
    time.sleep(5)    

['Albany', 2646121.0, 3100069.0, 4357419.0, 1548699.0, 2297530.0, 3871882.0, 1299103.0]
['Bronx', 8327787.0, 9578888.0, 14532777.0, 4588373.0, 7240064.0, 13670884.0, 4141447.0]
['Cattaraugus', 533540.0, 633977.0, 895314.0, 396573.0, 433440.0, 780493.0, 280295.0]
['Chautauqua', 881505.0, 1115182.0, 1629439.0, 637912.0, 712661.0, 1311482.0, 493576.0]
['Chenango', 336230.0, 410896.0, 585966.0, 248874.0, 273723.0, 469179.0, 173954.0]
['Columbia', 721055.0, 854399.0, 1190478.0, 540769.0, 611357.0, 1022223.0, 385143.0]
['Delaware', 358779.0, 430780.0, 667405.0, 281529.0, 301996.0, 521661.0, 204537.0]
['Erie', 7399749.0, 8984282.0, 12482678.0, 4837010.0, 6180184.0, 10533949.0, 3786312.0]
['Franklin', 414233.0, 472108.0, 663267.0, 268630.0, 352129.0, 599259.0, 206260.0]
['Genesee', 432748.0, 537061.0, 759552.0, 316481.0, 343751.0, 588556.0, 218883.0]
['Hamilton', 97283.0, 123569.0, 158639.0, 96126.0, 59300.0, 175626.0, 44229.0]
['Jefferson', 692184.0, 875151.0, 1147294.0, 479681.0, 526571.0, 9

In [125]:
# make into dataframe
incomeDF = pd.DataFrame(incomeData)

# add header
new_header = incomeDF.iloc[0] 
incomeDF = incomeDF[1:] 
incomeDF.columns = new_header 

# view
incomeDF

,County,Total (dollars):,Total Male (dollars):,"Total Male: Full-Time, Year-Round (dollars)",Total Male: Other (dollars),Total Female (dollars):,"Total Female: Full-Time, Year-Round (dollars)",Total Female: Other (dollars)
1,Albany,2646121.0,3100069.0,4357419.0,1548699.0,2297530.0,3871882.0,1299103.0
2,Allegany,294556.0,385827.0,585206.0,256544.0,220717.0,440929.0,154588.0
3,Bronx,8327787.0,9578888.0,14532777.0,4588373.0,7240064.0,13670884.0,4141447.0
4,Broome,1505106.0,1816483.0,2736401.0,1123739.0,1257354.0,2285045.0,830019.0
5,Cattaraugus,533540.0,633977.0,895314.0,396573.0,433440.0,780493.0,280295.0
...,...,...,...,...,...,...,...,...
59,Washington,505212.0,589364.0,854369.0,376208.0,429104.0,752961.0,286032.0
60,Wayne,683069.0,826521.0,1139451.0,482135.0,554349.0,889473.0,356275.0
61,Westchester,10730779.0,14535052.0,20591730.0,5425486.0,8278668.0,15465812.0,4249884.0
62,Wyoming,346071.0,439959.0,573114.0,285537.0,261366.0,436799.0,166505.0


In [126]:
# write to csv
# write to sex data to csv
# as backup
incomeDF.to_csv("NY_incomeData.csv")

In [127]:
# check csv saved correctly
# read data in from csv
# this is so you don't have to run through the previous for loop (saves time)

incomeDF = pd.read_csv('NY_incomeData.csv')
incomeDF = incomeDF.drop(['Unnamed: 0'], axis = 1)
incomeDF

,County,Total (dollars):,Total Male (dollars):,"Total Male: Full-Time, Year-Round (dollars)",Total Male: Other (dollars),Total Female (dollars):,"Total Female: Full-Time, Year-Round (dollars)",Total Female: Other (dollars)
0,Albany,2646121.0,3100069.0,4357419.0,1548699.0,2297530.0,3871882.0,1299103.0
1,Allegany,294556.0,385827.0,585206.0,256544.0,220717.0,440929.0,154588.0
2,Bronx,8327787.0,9578888.0,14532777.0,4588373.0,7240064.0,13670884.0,4141447.0
3,Broome,1505106.0,1816483.0,2736401.0,1123739.0,1257354.0,2285045.0,830019.0
4,Cattaraugus,533540.0,633977.0,895314.0,396573.0,433440.0,780493.0,280295.0
...,...,...,...,...,...,...,...,...
58,Washington,505212.0,589364.0,854369.0,376208.0,429104.0,752961.0,286032.0
59,Wayne,683069.0,826521.0,1139451.0,482135.0,554349.0,889473.0,356275.0
60,Westchester,10730779.0,14535052.0,20591730.0,5425486.0,8278668.0,15465812.0,4249884.0
61,Wyoming,346071.0,439959.0,573114.0,285537.0,261366.0,436799.0,166505.0


## Create Dataframe on Housing

In [ ]:
acs.filter_tables('HOUSE', by = 'description')

In [ ]:
totalHousingData = acs.filter_variables('B11016')

In [ ]:
incomeData = []

# columns to drop
to_drop = ['GEOID', 'geometry', 'state', 'county', 'tract', "NAME"]

# renamed column names
rename = ['Total (dollars):',
          'Total Male (dollars):',
          'Total Male: Full-Time, Year-Round (dollars)',
          'Total Male: Other (dollars)',
          'Total Female (dollars):',
          'Total Female: Full-Time, Year-Round (dollars)',
          'Total Female: Other (dollars)']


for i in range(len(counties)):

    name = counties[i] + ", NY"
    currentDF = acs.from_county(name, variables = 'B19326')
    current = counties[i]
    
    # if first round through
    # create labels to rename the dataframes
    if i == 0:
        indexVals = totalIncomeData.index
        labels = totalIncomeData['label'].to_list()
        labelsDict = OrderedDict({})
        for key in indexVals:
            for value in labels:
                labelsDict[key] = value
                labels.remove(value)
                break

        labelsDict.update({"geometry": "geometry"})
        labelsDict.move_to_end("geometry", last = False)

        labelsDict.update({"GEOID": "GEOID"})
        labelsDict.move_to_end("GEOID", last = False)

        labelsDict.update({"state": "state"})
        labelsDict.update({"county": "county"})
        labelsDict.update({"tract": "tract"})
        labelsDict.update({"NAME": "NAME"})
        
    
    # rename the dataframe
    currentDF.columns = currentDF.columns.to_series().map(labelsDict)
    
    # drop columns
    currentDF = currentDF.drop(to_drop, axis = 1)
    
    # rename the columns 
    currentDF = currentDF.set_axis(rename, axis = 1)
    
    
    # get the sums for each category and put into a dictonary
    current_sums = currentDF.sum().to_dict()
    current_sums = OrderedDict(current_sums)

    current_sums.update({"County": current})
    current_sums.move_to_end("County", last = False)

    current_sums = dict(current_sums)

    # if first run through
    # get the headers for the final dataframe
    # append to the final dataframe
    if i == 0:
        headers = list(current_sums.keys())
        incomeData.append(headers)
        
    # create inner_row to append
    # and append to final dataframe
    inner_row = list(current_sums.values())
    incomeData.append(inner_row)
    

    # for periodic checking
    if ((i % 2) == 0):
        print(inner_row)
        
        
        
    # force code to take time
    # do this to avoid a 429 error
    # (too many requests error)
    time.sleep(5)    